In [1]:
%reload_ext autoreload
%autoreload 2

import pickle
import sys

import numpy as np
import pandas as pd
from skimage.external import tifffile as T
from skimage import img_as_float

sys.path.append('diva-python/src/')
from train_rf import combining_RF
from utilities import create_df_features

### TRAINING 1st round

In [2]:
input_features = 'features_1er_tag.csv'
input_classifier = 'NONE'
output_classifier = "Model/rfc_31.pckl"

# initiate the class specifying path to features, name of the existing classifier and classifier strength
glob_class = combining_RF(input_features, input_classifier, 5)

# load features extracted on DIVA from tagged voxels
# and add log proba of the existing classifier to the features matrix
glob_class.__prepare_data_for_training__()

# fit the RFC using the features and store it in liste_learner
glob_class.__train_RF_classifier__()

# save the obtained classifier
pickle.dump(glob_class.liste_learner, open(output_classifier, 'wb'))

### INFERENCE 1st round

In [3]:
# open and load features for all the voxels of 1 packet in json format
input_features_file = 'packet7.json'
   
data_df = create_df_features(input_features_file)

In [4]:
classifier_filename = 'Model/rfc_31.pckl'

glob_class = combining_RF(data_df, classifier_filename, 5)
glob_class.__prepare_data_for_inference__()
proba = glob_class.__predict_proba__()

/home/corentin/anaconda3/envs/voxlea/lib/python3.7/site-packages/sklearn/ensemble/forest.py:635: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [5]:
np.save('Proba/rfc_31', proba)

In [6]:
proba = (1 - proba) * 255
proba_uint8 = proba.astype(np.uint8)

In [7]:
proba.shape, glob_class.features.shape

((474552,), (474552, 31))

### TRAINING 2nd round

In [8]:
input_features = 'features_2eme_tag.csv'
input_classifier = 'Model/rfc_31.pckl'
output_classifier = "Model/rfc_32.pckl"

# initiate the class specifying path to features, name of the existing classifier and classifier strength
glob_class = combining_RF(input_features, input_classifier, 5)

# load features extracted on DIVA from tagged voxels
# and add log proba of the existing classifier to the features matrix
glob_class.__prepare_data_for_training__()

# fit the RFC using the features and store it in liste_learner
glob_class.__train_RF_classifier__()

# save the obtained classifier
pickle.dump(glob_class.liste_learner, open(output_classifier, 'wb'))

/home/corentin/anaconda3/envs/voxlea/lib/python3.7/site-packages/sklearn/ensemble/forest.py:635: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


### INFERENCE 2nd round

In [9]:
classifier_filename = 'Model/rfc_32.pckl'

glob_class = combining_RF(data_df, classifier_filename, 5)
glob_class.__prepare_data_for_inference__()
proba = glob_class.__predict_proba__()

/home/corentin/anaconda3/envs/voxlea/lib/python3.7/site-packages/sklearn/ensemble/forest.py:635: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)
/home/corentin/anaconda3/envs/voxlea/lib/python3.7/site-packages/sklearn/ensemble/forest.py:635: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [10]:
np.save('Proba/rfc_32', proba)

In [11]:
proba = (1 - proba) * 255
proba_uint8 = proba.astype(np.uint8)

In [12]:
proba.shape, glob_class.features.shape

((474552,), (474552, 32))